In [8]:
import pandas as pd
import numpy as np
import time

import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
# beer_list = ['San Miguel',
# 'Newcastle Brown Ale',
# "Rickard's Red",
# 'Kaiser Beer',
# "Maclay's Traditional Pale Ale",
# 'Carlsberg',
# 'Weihenstephaner Hefeweissbier',
# 'Mill Street Original Organic Lager',
# 'Tatra Beer',
# 'Muskoka Cream Ale',
# 'Samuel Smiths Organic Lager',
# 'Cool Beer Blonde Lager',
# 'Keller Premium Lager',
# 'Creemore Springs Premium Lager',
# 'Erdinger Weissbier',
# 'Harp Lager',
# 'Paulaner Oktoberfest Bier (Wiesn Bier)',
# 'Great Lakes Brewery Canuck Pale Ale',
# 'DAB Original',
# 'Pilsner Urquell',
# 'Guinness Draught',
# 'Tsingtao Beer',
# 'Stella Artois']

In [9]:
beer_list = ["Heineken"]

In [10]:
len(beer_list)

1

In [11]:
beer_list = pd.DataFrame(data=beer_list, columns=['search_name'])
beer_list.head()

,search_name
0,Heineken


In [12]:
# Create DataFrame
data = pd.DataFrame(data=[], columns=['beer_info', 'search_name', 'beer_name'])
print(data)

# chromedriver.exe 
chromedriver = 'chromedriver.exe'
# URL
url = 'https://www.ratebeer.com/search?tab=beer'

# Web browser open using selenium
driver = webdriver.Chrome(chromedriver)
driver.get(url)
driver.set_window_size(900, 900)
time.sleep(1)

Empty DataFrame
Columns: [beer_info, search_name, beer_name]
Index: []


In [13]:
def crawl(driver, beer, data, k):
    # Create Data Frame
    data = pd.DataFrame(data=[], columns=['beer_info', 'search_name', 'beer_name'])
    print(data)

    # url open
    print('url_open... {0} We are collecting the beer data..'.format(beer))
    
    
    
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    driver.set_window_size(900, 900)

    
    # Step1 : Search Beer
    time.sleep(2)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)
    


    # Step2 : Select the Beer
    driver.find_element_by_xpath('//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()

    # Step3 : Scrapping the Product Beer
    time.sleep(3)
    beer_name = driver.find_element_by_css_selector('.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text

    error_cnt = 0

    while 1:
        try :
            # Step 4 : Scrapping all reviews (How many reviews do they have?)
            time.sleep(2)
            string = driver.find_element_by_class_name('MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text

            # checking if ',' is included
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            print('Success!... get out of while loop.')
            break
        except :
            print('Error..! Restarts.')

            error_cnt += 1

            if error_cnt == 5:
                print('Error is keep showing... Move to the next beer!')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # Step 5 : Click Score breakdown 
    time.sleep(3)
    element = driver.find_element_by_xpath('//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    time.sleep(3)
    # This is the code to move to the element. 
    driver.execute_script("arguments[0].click();", element)

    # calculate the number of the pages to check how many we need to scrap
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, 'Page')

        # Step 6 : scrapping all of the beer information
        time.sleep(3)
        beer_info = driver.find_elements_by_css_selector('.px-4.fj-s.f-wrap')

        tmp = []

        # save the data into dataframe 
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        tmp = pd.DataFrame(data=tmp, columns=['beer_info'])
        tmp['beer_name'] = beer_name
        tmp['search_name'] = beer
        data = pd.concat([data, tmp])

        # Step 7 : how to move to next page
        
        try :
            element = driver.find_element_by_xpath('//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(3)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('This is the last page')

    # cut the data by the number of the reviews if the data is duplicated
    if num != len(data):
        data = data[:num]

    print('The number of the reviews : ', num, 'The number of the reviews which are acutally collected : ', len(data))

    # Save as CSV, Excel 
    result = pd.merge(data, beer_list, on='search_name', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')
    result.to_excel("beer_n_"+str(k)+".xlsx")

    driver.quit()

    return result




In [7]:
# ForLoop - 1
for k in range(len(beer_list)):
    result = crawl(driver, beer_list['search_name'].iloc[k], data, k)

Empty DataFrame
Columns: [beer_info, search_name, beer_name]
Index: []
url_open... San Miguel We are collecting the beer data..
Success!... get out of while loop.
1 Page
2 Page
3 Page
4 Page
5 Page
6 Page
7 Page
8 Page
9 Page
10 Page
11 Page
12 Page
13 Page
The number of the reviews :  193 The number of the reviews which are acutally collected :  193
Empty DataFrame
Columns: [beer_info, search_name, beer_name]
Index: []
url_open... Newcastle Brown Ale We are collecting the beer data..
Success!... get out of while loop.
1 Page
2 Page
3 Page
4 Page
5 Page
6 Page
7 Page
8 Page
9 Page
10 Page
11 Page
12 Page
13 Page
14 Page
15 Page
16 Page
17 Page
18 Page
19 Page
20 Page
21 Page
22 Page
23 Page
24 Page
25 Page
26 Page
27 Page
28 Page
29 Page
30 Page
31 Page
32 Page
33 Page
34 Page
35 Page
36 Page
37 Page
38 Page
39 Page
40 Page
41 Page
42 Page
43 Page
44 Page
45 Page
46 Page
47 Page
48 Page
49 Page
50 Page
51 Page
52 Page
53 Page
54 Page
55 Page
56 Page
57 Page
58 Page
59 Page
60 Page
61 P

9 Page
10 Page
11 Page
12 Page
13 Page
14 Page
15 Page
16 Page
17 Page
18 Page
19 Page
20 Page
21 Page
22 Page
23 Page
24 Page
25 Page
26 Page
27 Page
28 Page
29 Page
30 Page
31 Page
32 Page
33 Page
34 Page
The number of the reviews :  501 The number of the reviews which are acutally collected :  501
Empty DataFrame
Columns: [beer_info, search_name, beer_name]
Index: []
url_open... Muskoka Cream Ale We are collecting the beer data..
Success!... get out of while loop.
1 Page
2 Page
3 Page
4 Page
5 Page
6 Page
7 Page
8 Page
9 Page
10 Page
11 Page
12 Page
13 Page
14 Page
15 Page
16 Page
The number of the reviews :  237 The number of the reviews which are acutally collected :  237
Empty DataFrame
Columns: [beer_info, search_name, beer_name]
Index: []
url_open... Samuel Smiths Organic Lager We are collecting the beer data..
Success!... get out of while loop.
1 Page
2 Page
3 Page
4 Page
5 Page
6 Page
7 Page
8 Page
9 Page
10 Page
11 Page
12 Page
13 Page
14 Page
15 Page
16 Page
17 Page
18 Page

48 Page
49 Page
50 Page
51 Page
52 Page
53 Page
54 Page
55 Page
56 Page
57 Page
58 Page
59 Page
60 Page
61 Page
62 Page
63 Page
64 Page
65 Page
66 Page
67 Page
68 Page
69 Page
70 Page
71 Page
72 Page
73 Page
74 Page
75 Page
76 Page
77 Page
78 Page
79 Page
80 Page
81 Page
82 Page
83 Page
84 Page
85 Page
86 Page
87 Page
88 Page
89 Page
90 Page
91 Page
92 Page
93 Page
94 Page
95 Page
96 Page
97 Page
98 Page
99 Page
100 Page
101 Page
102 Page
103 Page
104 Page
105 Page
106 Page
107 Page
108 Page
109 Page
110 Page
111 Page
112 Page
113 Page
114 Page
115 Page
116 Page
117 Page
118 Page
119 Page
120 Page
121 Page
122 Page
123 Page
124 Page
125 Page
126 Page
127 Page
128 Page
129 Page
130 Page
131 Page
132 Page
133 Page
134 Page
135 Page
136 Page
137 Page
138 Page
139 Page
140 Page
141 Page
142 Page
143 Page
144 Page
145 Page
146 Page
147 Page
148 Page
149 Page
150 Page
151 Page
152 Page
153 Page
154 Page
155 Page
156 Page
157 Page
158 Page
159 Page
160 Page
161 Page
162 Page
163 Page
164 Page

229 Page
230 Page
231 Page
232 Page
233 Page
234 Page
235 Page
236 Page
237 Page
238 Page
239 Page
240 Page
241 Page
The number of the reviews :  3610 The number of the reviews which are acutally collected :  3610


In [16]:
k = 24
result = crawl(driver, "Heineken", data, k)

Empty DataFrame
Columns: [beer_info, search_name, beer_name]
Index: []
url_open... Heineken We are collecting the beer data..
Success!... get out of while loop.
1 Page
2 Page
3 Page
4 Page
5 Page
6 Page
7 Page
8 Page
9 Page
10 Page
11 Page
12 Page
13 Page
14 Page
15 Page
16 Page
17 Page
18 Page
19 Page
20 Page
21 Page
22 Page
23 Page
24 Page
25 Page
26 Page
27 Page
28 Page
29 Page
30 Page
31 Page
32 Page
33 Page
34 Page
35 Page
36 Page
37 Page
38 Page
39 Page
40 Page
41 Page
42 Page
43 Page
44 Page
45 Page
46 Page
47 Page
48 Page
49 Page
50 Page
51 Page
52 Page
53 Page
54 Page
55 Page
56 Page
57 Page
58 Page
59 Page
60 Page
61 Page
62 Page
63 Page
64 Page
65 Page
66 Page
67 Page
68 Page
69 Page
70 Page
71 Page
72 Page
73 Page
74 Page
75 Page
76 Page
77 Page
78 Page
79 Page
80 Page
81 Page
82 Page
83 Page
84 Page
85 Page
86 Page
87 Page
88 Page
89 Page
90 Page
91 Page
92 Page
93 Page
94 Page
95 Page
96 Page
97 Page
98 Page
99 Page
100 Page
101 Page
102 Page
103 Page
104 Page
105 Page
10